### 

In [13]:
import pandas as pd
import numpy as np
import datetime
from datetime import datetime, timedelta
import utils


In [14]:
events = utils.get_events()
events.head()

,timestamp,event,person,url,sku,model,condition,storage,color,skus,...,city,region,country,device_type,screen_resolution,operating_system_version,browser_version,month,day,hora
0,2018-05-18 00:11:59,viewed product,4886f805,NaN,9288.0,Samsung Galaxy J7 Prime,Excelente,32GB,Dourado,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5,18,0
1,2018-05-18 00:11:27,viewed product,ad93850f,NaN,304.0,iPhone 5s,Muito Bom,32GB,Cinza espacial,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5,18,0
2,2018-05-18 00:11:16,viewed product,0297fc1e,NaN,6888.0,iPhone 6S,Muito Bom,64GB,Prateado,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5,18,0
3,2018-05-18 00:11:14,viewed product,2d681dd8,NaN,11890.0,iPhone 7,Bom,128GB,Vermelho,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5,18,0
4,2018-05-18 00:11:09,viewed product,cccea85e,NaN,7517.0,LG G4 H818P,Excelente,32GB,Branco,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5,18,0


In [15]:
df_viewed_checkout = events[(events['event'] == 'viewed product') | ((events['event'] == 'checkout'))]
df_viewed_checkout = df_viewed_checkout[['timestamp', 'person', 'event', 'sku']]
df_viewed_checkout.head()

,timestamp,person,event,sku
0,2018-05-18 00:11:59,4886f805,viewed product,9288.0
1,2018-05-18 00:11:27,ad93850f,viewed product,304.0
2,2018-05-18 00:11:16,0297fc1e,viewed product,6888.0
3,2018-05-18 00:11:14,2d681dd8,viewed product,11890.0
4,2018-05-18 00:11:09,cccea85e,viewed product,7517.0


In [16]:
df_viewed_checkout = df_viewed_checkout.groupby(['person', 'event', 'sku']).agg({'timestamp':'count'})
df_viewed_checkout = df_viewed_checkout.reset_index()
df_viewed_checkout.head()

,person,event,sku,timestamp
0,0008ed71,checkout,3372.0,1
1,0008ed71,checkout,7505.0,1
2,0008ed71,checkout,8247.0,1
3,00091926,checkout,6901.0,1
4,00091926,checkout,6930.0,1


In [19]:
df_viewed_checkout = df_viewed_checkout.groupby(['person','sku','event'])['timestamp'].count().unstack().fillna(0)
df_viewed_checkout.head()

event            checkout  viewed product
person   sku                             
0008ed71 3372.0       1.0             0.0
         7505.0       1.0             0.0
         8247.0       1.0             0.0
00091926 1365.0       0.0             1.0
         1373.0       0.0             1.0

In [20]:
index_df = df_viewed_checkout.index.to_frame(index=False)
df_viewed_checkout = df_viewed_checkout.reset_index(drop=True)
df_viewed_checkout = pd.merge(index_df, df_viewed_checkout, left_index=True, right_index=True)
df_viewed_checkout.head()

,person,sku,checkout,viewed product
0,0008ed71,3372.0,1.0,0.0
1,0008ed71,7505.0,1.0,0.0
2,0008ed71,8247.0,1.0,0.0
3,00091926,1365.0,0.0,1.0
4,00091926,1373.0,0.0,1.0


In [21]:
df_viewed_checkout['viewed/checkouted'] = df_viewed_checkout['checkout']/df_viewed_checkout['viewed product']
df_viewed_checkout['viewed/checkouted'] = df_viewed_checkout['viewed/checkouted'].replace(np.inf, -1)
df_viewed_checkout.head()

,person,sku,checkout,viewed product,viewed/checkouted
0,0008ed71,3372.0,1.0,0.0,-1.0
1,0008ed71,7505.0,1.0,0.0,-1.0
2,0008ed71,8247.0,1.0,0.0,-1.0
3,00091926,1365.0,0.0,1.0,0.0
4,00091926,1373.0,0.0,1.0,0.0


In [22]:
df_viewed_checkout = df_viewed_checkout.groupby('person').agg({'viewed/checkouted':('max','min'), 'checkout':'sum', 'viewed product':'sum'})
df_viewed_checkout.head()

viewed/checkouted      checkout viewed product
                       max  min      sum            sum
person                                                 
0008ed71              -1.0 -1.0      3.0            0.0
00091926               1.0  0.0      2.0          164.0
00091a7a               0.0  0.0      0.0            3.0
000ba417               1.0  0.0      4.0           81.0
000c79fe               1.0  1.0      1.0            1.0

In [23]:
df_viewed_checkout=df_viewed_checkout.reset_index()
df_viewed_checkout.columns = ['person','viewed/checkouted min','viewed/checkouted max','checkout sum','viewed sum']
df_viewed_checkout.head()

,person,viewed/checkouted min,viewed/checkouted max,checkout sum,viewed sum
0,0008ed71,-1.0,-1.0,3.0,0.0
1,00091926,1.0,0.0,2.0,164.0
2,00091a7a,0.0,0.0,0.0,3.0
3,000ba417,1.0,0.0,4.0,81.0
4,000c79fe,1.0,1.0,1.0,1.0


In [24]:
df_viewed_checkout['viewed/checkout gral'] = df_viewed_checkout['viewed sum']/df_viewed_checkout['checkout sum']
df_viewed_checkout['viewed/checkout gral'] = df_viewed_checkout['viewed/checkout gral'].replace(np.inf, -1)
df_viewed_checkout = df_viewed_checkout.drop(['checkout sum', 'viewed sum'], axis=1)

In [25]:
df_viewed_checkout.head()

,person,viewed/checkouted min,viewed/checkouted max,viewed/checkout gral
0,0008ed71,-1.0,-1.0,0.00
1,00091926,1.0,0.0,82.00
2,00091a7a,0.0,0.0,-1.00
3,000ba417,1.0,0.0,20.25
4,000c79fe,1.0,1.0,1.00


In [26]:
df_viewed_checkout.to_csv('features_productos_checkouts.csv',index=False)